In [55]:
import pandas as pd
import numpy as np

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM, Input, InputLayer, Dropout, Dense, Flatten, Embedding, Add, Concatenate
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import RMSprop

import torch
from sklearn.metrics import precision_recall_fscore_support

In [2]:
# Good data
# df = pd.read_csv('./data/kokil dec 6 reprepare/affcon_final_with_linguistics.csv')

In [3]:
# Messy data
# df = pd.read_csv('./data/affcon_final_politeness_strategies_merge.csv')

In [65]:
# Data with Throughput & WorkTime
df = pd.read_csv('./data/kokil dec 6 reprepare/conf_pc_worker_sem.csv')

In [58]:
df = df.dropna()

In [64]:
len(df)

4013

In [4]:
df_small = df[0:500]
df = df_small

In [5]:
len(df)

500

In [4]:
max(df_throughput.max())

NameError: name 'df_throughput' is not defined

In [6]:
# Throughput
df_throughput = df[['Throughput.1', 'Throughput.2', 'Throughput.3', 'Throughput.4', 'Throughput.5']].copy()
df_throughput['avg_throughput'] = df_throughput.mean(axis=1) / max(df_throughput.max())

# WorkTime
df_worktime = df[['WorkTime.1', 'WorkTime.2', 'WorkTime.3', 'WorkTime.4', 'WorkTime.5']].copy()
df_worktime['avg_worktime'] = df_worktime.mean(axis=1) / max(df_worktime.max())

In [7]:
indices = range(df.shape[0])

### Individual Classifiers are LSTMS

In [8]:
train, test, indices_train, indices_test = train_test_split(df, indices, test_size=0.2)

new_deception_train = train["Input.deception_quadrant"].copy()
new_deception_train['Input.deception_quadrant'] = train["Input.deception_quadrant"].apply(lambda x : 1 if x == "Straightforward" else 0)
y_train_deception = new_deception_train['Input.deception_quadrant'].tolist()
y_train_rapport = train['Answer.3rapport.yes_label'].tolist()
y_train_share_information = train['Answer.4shareinformation.yes_label'].tolist()
y_train_reasoning = train['Answer.2reasoning.yes_label'].tolist()
y_train_gamemove = train['Answer.1gamemove.yes_label'].tolist()

X_train_col = train['Input.full_text']

new_deception_test = test["Input.deception_quadrant"].copy()
new_deception_test['Input.deception_quadrant'] = test["Input.deception_quadrant"].apply(lambda x : 1 if x == "Straightforward" else 0)
y_test_deception = new_deception_test['Input.deception_quadrant'].tolist()
y_test_rapport = test['Answer.3rapport.yes_label'].tolist()
y_test_share_information = test['Answer.4shareinformation.yes_label'].tolist()
y_test_reasoning = test['Answer.2reasoning.yes_label'].tolist()
y_test_gamemove = test['Answer.1gamemove.yes_label'].tolist()

X_test_col = test['Input.full_text']

In [9]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto',
    baseline=None, restore_best_weights=False
)

In [10]:
le = LabelEncoder()

y_train_deception = le.fit_transform(y_train_deception)
y_train_deception = y_train_deception.reshape(-1,1)

y_train_rapport = le.fit_transform(y_train_rapport)
y_train_rapport = y_train_rapport.reshape(-1,1)

y_train_share_information = le.fit_transform(y_train_share_information)
y_train_share_information = y_train_share_information.reshape(-1,1)

y_train_reasoning = le.fit_transform(y_train_reasoning)
y_train_reasoning = y_train_reasoning.reshape(-1,1)

y_train_gamemove = le.fit_transform(y_train_gamemove)
y_train_gamemove = y_train_gamemove.reshape(-1,1)

y_train_deception = le.fit_transform(y_train_deception)
y_train_deception = y_train_deception.reshape(-1,1)

y_test_rapport = le.fit_transform(y_test_rapport)
y_test_rapport = y_test_rapport.reshape(-1,1)

y_test_share_information = le.fit_transform(y_test_share_information)
y_test_share_information = y_test_share_information.reshape(-1,1)

y_test_reasoning = le.fit_transform(y_test_reasoning)
y_test_reasoning = y_test_reasoning.reshape(-1,1)

y_test_gamemove = le.fit_transform(y_test_gamemove)
y_test_gamemove = y_test_gamemove.reshape(-1,1)

y_test_deception = le.fit_transform(y_test_deception)
y_test_deception = y_test_deception.reshape(-1,1)

C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [11]:
max_words = 1000
max_len = 220

tok = Tokenizer(num_words=max_words)

tok.fit_on_texts(X_train_col)
X_train_sequences = tok.texts_to_sequences(X_train_col)
X_train = pad_sequences(X_train_sequences, maxlen=max_len)

tok.fit_on_texts(X_test_col)
X_test_sequences = tok.texts_to_sequences(X_test_col)
X_test = pad_sequences(X_test_sequences, maxlen=max_len)

In [12]:
early_stop = EarlyStopping(monitor='val_loss',min_delta=0.00001)
def create_lstm():
    Inp = Input(name='inputs', shape=[max_len])
    x = Embedding(max_words, 50, input_length=max_len)(Inp)
    x = LSTM(64, name='LSTM_01')(x)
    x = Dropout(0.5, name='Dropout')(x)
    x = Dense(128, activation='relu',name='Dense_01')(x)
    # x = Dropout(0.5,name='Dropout')(x)
    out = Dense(1,activation='sigmoid', name='output')(x)
    model = Model(inputs=Inp, outputs=out)
    
    return model

### Construct individual LSTM models

In [13]:
# Rapport model
rapport_model = create_lstm()
rapport_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', f1_m, recall_m, precision_m])
rapport_model.fit(X_train,y_train_rapport,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_rapport), callbacks=[early_stop])

Epoch 1/15
4/4 [==============================] - 1s 172ms/step - loss: 0.6746 - accuracy: 0.6450 - f1_m: 0.7314 - recall_m: 0.7705 - precision_m: 0.8510 - val_loss: 0.3908 - val_accuracy: 0.8700 - val_f1_m: 0.9305 - val_recall_m: 1.0000 - val_precision_m: 0.8700
Epoch 2/15
4/4 [==============================] - 0s 29ms/step - loss: 0.4664 - accuracy: 0.8575 - f1_m: 0.9324 - recall_m: 1.0000 - precision_m: 0.8750 - val_loss: 0.4220 - val_accuracy: 0.8700 - val_f1_m: 0.9305 - val_recall_m: 1.0000 - val_precision_m: 0.8700


In [14]:
rapport_pred = rapport_model.predict(X_train)
rapport_pred_test = rapport_model.predict(X_test)

In [15]:
rapport_pred_test_round = rapport_pred_test.round()
precision_recall_fscore_support(y_test_rapport, rapport_pred_test_round, average='macro')

C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.435, 0.5, 0.4652406417112299, None)

In [16]:
# Game move model
gamemove_model = create_lstm()
gamemove_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', f1_m, recall_m, precision_m])
gamemove_model.fit(X_train,y_train_gamemove,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_gamemove), callbacks=[early_stop])

Epoch 1/15
4/4 [==============================] - 1s 162ms/step - loss: 0.6612 - accuracy: 0.7925 - f1_m: 0.8918 - recall_m: 0.8969 - precision_m: 0.9022 - val_loss: 0.2544 - val_accuracy: 0.9300 - val_f1_m: 0.9637 - val_recall_m: 1.0000 - val_precision_m: 0.9300
Epoch 2/15
4/4 [==============================] - 0s 29ms/step - loss: 0.3538 - accuracy: 0.9025 - f1_m: 0.9529 - recall_m: 1.0000 - precision_m: 0.9102 - val_loss: 0.2550 - val_accuracy: 0.9300 - val_f1_m: 0.9637 - val_recall_m: 1.0000 - val_precision_m: 0.9300


In [17]:
gamemove_pred = gamemove_model.predict(X_train)
gamemove_pred_test = gamemove_model.predict(X_test)

In [18]:
gamemove_pred_test_round = gamemove_pred_test.round()
precision_recall_fscore_support(y_test_gamemove, gamemove_pred_test_round, average='macro')

(0.465, 0.5, 0.48186528497409326, None)

In [19]:
# Reasoning model
reasoning_model = create_lstm()
reasoning_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', f1_m, recall_m, precision_m])
reasoning_model.fit(X_train,y_train_reasoning,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_reasoning), callbacks=[early_stop])

Epoch 1/15
4/4 [==============================] - 1s 160ms/step - loss: 0.6649 - accuracy: 0.7750 - f1_m: 0.8817 - recall_m: 0.9393 - precision_m: 0.8337 - val_loss: 0.5271 - val_accuracy: 0.8100 - val_f1_m: 0.8950 - val_recall_m: 1.0000 - val_precision_m: 0.8100
Epoch 2/15
4/4 [==============================] - 0s 28ms/step - loss: 0.4879 - accuracy: 0.8275 - f1_m: 0.9115 - recall_m: 1.0000 - precision_m: 0.8379 - val_loss: 0.4899 - val_accuracy: 0.8100 - val_f1_m: 0.8950 - val_recall_m: 1.0000 - val_precision_m: 0.8100
Epoch 3/15
4/4 [==============================] - 0s 33ms/step - loss: 0.4629 - accuracy: 0.8275 - f1_m: 0.9036 - recall_m: 1.0000 - precision_m: 0.8242 - val_loss: 0.4875 - val_accuracy: 0.8100 - val_f1_m: 0.8950 - val_recall_m: 1.0000 - val_precision_m: 0.8100
Epoch 4/15
4/4 [==============================] - 0s 31ms/step - loss: 0.4642 - accuracy: 0.8275 - f1_m: 0.9114 - recall_m: 1.0000 - precision_m: 0.8379 - val_loss: 0.4889 - val_accuracy: 0.8100 - val_f1_m: 0.8

In [20]:
reasoning_pred = reasoning_model.predict(X_train)
reasoning_pred_test = reasoning_model.predict(X_test)

In [21]:
reasoning_pred_test_round = reasoning_pred_test.round()
precision_recall_fscore_support(y_test_reasoning, reasoning_pred_test_round, average='macro')

(0.405, 0.5, 0.44751381215469616, None)

In [22]:
# Share Information model
shareinfo_model = create_lstm()
shareinfo_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', f1_m, recall_m, precision_m])
shareinfo_model.fit(X_train,y_train_share_information,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_share_information), callbacks=[early_stop])

Epoch 1/15
4/4 [==============================] - 1s 172ms/step - loss: 0.6718 - accuracy: 0.7400 - f1_m: 0.8375 - recall_m: 0.8915 - precision_m: 0.8170 - val_loss: 0.5294 - val_accuracy: 0.7900 - val_f1_m: 0.8827 - val_recall_m: 1.0000 - val_precision_m: 0.7900
Epoch 2/15
4/4 [==============================] - 0s 29ms/step - loss: 0.4914 - accuracy: 0.8225 - f1_m: 0.9090 - recall_m: 1.0000 - precision_m: 0.8340 - val_loss: 0.5706 - val_accuracy: 0.7900 - val_f1_m: 0.8827 - val_recall_m: 1.0000 - val_precision_m: 0.7900


In [23]:
shareinfo_pred = shareinfo_model.predict(X_train)
shareinfo_pred_test = shareinfo_model.predict(X_test)

In [24]:
shareinfo_pred_test_round = shareinfo_pred_test.round()
precision_recall_fscore_support(y_test_share_information, shareinfo_pred_test_round, average='macro')

(0.395, 0.5, 0.44134078212290506, None)

In [25]:
# Deception model
deception_model = create_lstm()
deception_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', f1_m, recall_m, precision_m])
deception_model.fit(X_train,y_train_deception,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_deception), callbacks=[early_stop])

Epoch 1/15
4/4 [==============================] - 1s 263ms/step - loss: 0.6604 - accuracy: 0.8450 - f1_m: 0.9184 - recall_m: 0.8770 - precision_m: 1.0000 - val_loss: 1.5548 - val_accuracy: 0.0000e+00 - val_f1_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_precision_m: 0.0000e+00
Epoch 2/15
4/4 [==============================] - 0s 28ms/step - loss: 0.1139 - accuracy: 0.9950 - f1_m: 0.9980 - recall_m: 1.0000 - precision_m: 0.9961 - val_loss: 4.9670 - val_accuracy: 0.0000e+00 - val_f1_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_precision_m: 0.0000e+00


In [26]:
deception_pred = deception_model.predict(X_train)
deception_pred_test = deception_model.predict(X_test)
deception_pred_test_round = deception_pred_test.round()
precision_recall_fscore_support(y_test_deception, deception_pred_test_round, average='macro')

C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.0, 0.0, 0.0, None)

### One hot encodings

In [27]:
# Train encodings
pred_df_arr_full = []
pred_df_arr = []
for i in range(0, len(gamemove_pred)):
    pred_obj_1 = {}
    pred_obj_1['gamemove'] = gamemove_pred[i][0]
    pred_obj_1['reasoning'] = reasoning_pred[i][0]
    pred_obj_1['shareinfo'] = shareinfo_pred[i][0]
    pred_df_arr.append(pred_obj_1)
    
    pred_obj_2 = pred_obj_1.copy()
    pred_obj_2['rapport'] = rapport_pred[i][0]
    pred_df_arr_full.append(pred_obj_2)
    
pred_df_full = pd.DataFrame(pred_df_arr_full)
pred_df = pd.DataFrame(pred_df_arr)

In [28]:
# Test encodings
pred_test_df_arr_full = []
pred_test_df_arr = []

for i in range(0, len(gamemove_pred_test)):
    pred_obj_1 = {}
    pred_obj_1['gamemove'] = gamemove_pred_test[i][0]
    pred_obj_1['reasoning'] = reasoning_pred_test[i][0]
    pred_obj_1['shareinfo'] = shareinfo_pred_test[i][0]
    pred_test_df_arr.append(pred_obj_1)
    
    pred_obj_2 = pred_obj_1.copy()
    pred_obj_2['rapport'] = rapport_pred_test[i][0]
    pred_test_df_arr_full.append(pred_obj_2)
    
pred_test_df_full = pd.DataFrame(pred_test_df_arr_full)
pred_test_df = pd.DataFrame(pred_test_df_arr)

### Joint Model with one hot encoding 

In [41]:
def create_joint_model(df):
    inputB = Input(shape=(df.shape[1],))
    c = Dense(2, activation='relu')(inputB)
    c = Dense(4, activation='relu')(c)
    c = Dense(2, activation='softmax')(c)
    full_model = Model(inputs=inputB, outputs=c)

    full_model.compile(loss='binary_crossentropy', optimizer='adam', 
                          metrics=['acc',f1_m,precision_m, recall_m])
    
    return full_model

In [42]:
print('Joint full model with one hot encoding, predicting deception')
joint_full_model = create_joint_model(pred_df_full)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_full, y=y_train_deception, epochs=32, 
                    batch_size=64, 
                    validation_data=(pred_test_df_full,y_test_deception), callbacks=[callback])

Joint full model with one hot encoding, predicting deception
Model: "functional_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 10        
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 12        
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 10        
Total params: 32
Trainable params: 32
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
7/7 [==============================] - 0s 40ms/step - loss: 0.8845 - acc: 0.0025 - f1_m: 0.9989 - precision_m: 0.9978 - recall_m: 1.0000 - val_loss: 0.8654 - val_acc: 0.0100 - val_f1_

Epoch 30/32
7/7 [==============================] - 0s 6ms/step - loss: 0.6933 - acc: 0.0400 - f1_m: 0.9989 - precision_m: 0.9978 - recall_m: 1.0000 - val_loss: 0.6932 - val_acc: 0.0300 - val_f1_m: 0.9961 - val_precision_m: 0.9922 - val_recall_m: 1.0000
Epoch 31/32
7/7 [==============================] - 0s 6ms/step - loss: 0.6932 - acc: 0.0600 - f1_m: 0.9989 - precision_m: 0.9978 - recall_m: 1.0000 - val_loss: 0.6932 - val_acc: 0.0500 - val_f1_m: 0.9961 - val_precision_m: 0.9922 - val_recall_m: 1.0000
Epoch 32/32
7/7 [==============================] - 0s 6ms/step - loss: 0.6932 - acc: 0.0775 - f1_m: 0.9989 - precision_m: 0.9978 - recall_m: 1.0000 - val_loss: 0.6932 - val_acc: 0.0500 - val_f1_m: 0.9961 - val_precision_m: 0.9922 - val_recall_m: 1.0000


In [44]:
joint_predict = joint_full_model.predict(pred_test_df_full)
joint_predict_round = []
for a in joint_predict:
    joint_predict_round.append([np.argmax(a)])
precision_recall_fscore_support(y_test_deception, np.array(joint_predict_round), average='macro')

(0.5052083333333334, 0.5202020202020202, 0.049144229806826145, None)

In [45]:
print('Joint full model with one hot encoding, predicting rapport')
joint_full_model = create_joint_model(pred_df)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df, y=y_train_rapport, epochs=32, 
                    batch_size=64, 
                    validation_data=(pred_test_df,y_test_rapport), callbacks=[callback])

Joint full model with one hot encoding, predicting rapport
Model: "functional_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 8         
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 12        
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 10        
Total params: 30
Trainable params: 30
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
7/7 [==============================] - 0s 37ms/step - loss: 0.9949 - acc: 0.8750 - f1_m: 0.9329 - precision_m: 0.8750 - recall_m: 1.0000 - val_loss: 0.9730 - val_acc: 0.8000 - val_f1_m:

Epoch 30/32
7/7 [==============================] - 0s 7ms/step - loss: 0.6944 - acc: 0.8750 - f1_m: 0.9249 - precision_m: 0.8616 - recall_m: 1.0000 - val_loss: 0.6943 - val_acc: 0.8000 - val_f1_m: 0.8858 - val_precision_m: 0.7951 - val_recall_m: 1.0000
Epoch 31/32
7/7 [==============================] - 0s 7ms/step - loss: 0.6941 - acc: 0.8750 - f1_m: 0.9367 - precision_m: 0.8817 - recall_m: 1.0000 - val_loss: 0.6940 - val_acc: 0.8000 - val_f1_m: 0.8858 - val_precision_m: 0.7951 - val_recall_m: 1.0000
Epoch 32/32
7/7 [==============================] - 0s 7ms/step - loss: 0.6939 - acc: 0.8750 - f1_m: 0.9197 - precision_m: 0.8549 - recall_m: 1.0000 - val_loss: 0.6938 - val_acc: 0.8000 - val_f1_m: 0.8858 - val_precision_m: 0.7951 - val_recall_m: 1.0000


In [46]:
joint_predict = joint_full_model.predict(pred_test_df)
joint_predict_round = []
for a in joint_predict:
    joint_predict_round.append([np.argmax(a)])
precision_recall_fscore_support(y_test_rapport, np.array(joint_predict_round), average='macro')

C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.4, 0.5, 0.4444444444444445, None)

### Joint Model by Keras Concatenate

In [26]:
def create_concatenate_keras_model(models_arr):
    commonInput = Input(shape=[max_len])

    input_model_arr = []
    for model in models_arr: 
        outmodel = model(commonInput)
        input_model_arr.append(outmodel)
    
    mergedOut = Concatenate()(input_model_arr)

    mergedOut = Flatten()(mergedOut)    
    mergedOut = Dense(256, activation='relu')(mergedOut)
    mergedOut = Dropout(.5)(mergedOut)
    mergedOut = Dense(128, activation='relu')(mergedOut)
    mergedOut = Dropout(.35)(mergedOut)
    mergedOut = Dense(2, activation='softmax')(mergedOut)  #Cuz binary

    mergedModel = Model(commonInput, mergedOut)
    mergedModel.compile(loss='binary_crossentropy', optimizer=RMSprop(), metrics=['accuracy', f1_m, recall_m, precision_m])
    
    return mergedModel

In [27]:
print('Joint model by concatenate, predicting deception')
merged_model = create_concatenate_keras_model([gamemove_model, reasoning_model, shareinfo_model, rapport_model])
merged_model.fit(X_train,y_train_deception,
          batch_size=128,
          epochs=15,
          validation_data=(X_test,y_test_deception), callbacks=[early_stop])

Joint model by concatenate, predicting deception
Epoch 1/15
72/72 [==============================] - 6s 80ms/step - loss: 0.6942 - accuracy: 0.5033 - f1_m: 0.9751 - recall_m: 1.0000 - precision_m: 0.9517 - val_loss: 0.6931 - val_accuracy: 0.9503 - val_f1_m: 0.9744 - val_recall_m: 1.0000 - val_precision_m: 0.9502
Epoch 2/15
72/72 [==============================] - 5s 68ms/step - loss: 0.6931 - accuracy: 0.5021 - f1_m: 0.9732 - recall_m: 0.9999 - precision_m: 0.9483 - val_loss: 0.6931 - val_accuracy: 0.9503 - val_f1_m: 0.9744 - val_recall_m: 1.0000 - val_precision_m: 0.9502


In [28]:
print('Joint model by concatenate, predicting rapport')
merged_model = create_concatenate_keras_model([gamemove_model, reasoning_model, shareinfo_model])
merged_model.fit(X_train,y_train_rapport,
          batch_size=128,
          epochs=15,
          validation_data=(X_test,y_test_rapport), callbacks=[early_stop])

Joint model by concatenate, predicting rapport
Epoch 1/15
72/72 [==============================] - 4s 62ms/step - loss: 0.6932 - accuracy: 0.4965 - f1_m: 0.9278 - recall_m: 1.0000 - precision_m: 0.8660 - val_loss: 0.6931 - val_accuracy: 0.8562 - val_f1_m: 0.9223 - val_recall_m: 1.0000 - val_precision_m: 0.8563
Epoch 2/15
72/72 [==============================] - 4s 52ms/step - loss: 0.6931 - accuracy: 0.5045 - f1_m: 0.9279 - recall_m: 1.0000 - precision_m: 0.8660 - val_loss: 0.6931 - val_accuracy: 0.8562 - val_f1_m: 0.9223 - val_recall_m: 1.0000 - val_precision_m: 0.8563


## Weighted against Throughput & WorkTime

In [29]:
# Train weighted encodings
pred_df_full_throughput = pred_df_full.copy()
pred_df_full_worktime = pred_df_full.copy()
pred_df_throughput = pred_df.copy()
pred_df_worktime = pred_df.copy()

throughput_values = df_throughput['avg_throughput'].take(indices_train).values
pred_df_full_throughput = pred_df_full_throughput.mul(throughput_values, axis=0)
pred_df_throughput = pred_df_throughput.mul(throughput_values, axis=0)

worktime_values = df_worktime['avg_worktime'].take(indices_train).values
pred_df_full_worktime = pred_df_full_worktime.mul(worktime_values, axis=0)
pred_df_worktime = pred_df_worktime.mul(worktime_values, axis=0)

In [30]:
# Test weighted encodings
pred_df_full_throughput_test = pred_test_df_full.copy()
pred_df_full_worktime_test = pred_test_df_full.copy()
pred_df_throughput_test = pred_test_df.copy()
pred_df_worktime_test = pred_test_df.copy()

throughput_values_test = df_throughput['avg_throughput'].take(indices_test).values
pred_df_full_throughput_test = pred_df_full_throughput_test.mul(throughput_values_test, axis=0)
pred_df_throughput_test = pred_df_throughput_test.mul(throughput_values_test, axis=0)

worktime_values_test = df_worktime['avg_worktime'].take(indices_test).values
pred_df_full_worktime_test = pred_df_full_worktime_test.mul(worktime_values_test, axis=0)
pred_df_worktime_test = pred_df_worktime_test.mul(worktime_values_test, axis=0)

## Throughput only

### Joint Model with one hot encoding 

In [31]:
def create_joint_model(df):
    inputB = Input(shape=(df.shape[1],))
    c = Dense(2, activation='relu')(inputB)
    c = Dense(4, activation='relu')(c)
    c = Dense(2, activation='softmax')(c)
    full_model = Model(inputs=inputB, outputs=c)

    full_model.compile(loss='binary_crossentropy', optimizer='adam', 
                          metrics=['acc',f1_m,precision_m, recall_m])
    
    return full_model

In [46]:
print('Joint full model with one hot encoding, predicting deception, weighted by throughput')
joint_full_model = create_joint_model(pred_df_full_throughput)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_full_throughput, y=y_train_deception, epochs=32, 
                    batch_size=64, 
                    validation_data=(pred_df_full_throughput_test,y_test_deception), callbacks=[callback])

Joint full model with one hot encoding, predicting deception, weighted by throughput
Model: "functional_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense_21 (Dense)             (None, 2)                 10        
_________________________________________________________________
dense_22 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_23 (Dense)             (None, 2)                 10        
Total params: 32
Trainable params: 32
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
7/7 [==============================] - 0s 36ms/step - loss: 0.6936 - acc: 0.9875 - f1_m: 0.9978 - precision_m: 0.9955 - recall_m: 1.0000 - val_loss: 0.6934 - v

In [47]:
joint_predict = joint_full_model.predict(pred_df_full_throughput_test)
joint_predict_round = []
for a in joint_predict:
    joint_predict_round.append([np.argmax(a)])
precision_recall_fscore_support(y_test_deception, np.array(joint_predict_round), average='macro')

C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.5, 0.34, 0.40476190476190477, None)

In [48]:
y_test_rapport = np.asarray(y_test_rapport)
y_train_rapport = np.asarray(y_train_rapport)

In [53]:
print('Joint full model with one hot encoding, predicting rapport, weighted by throughput')
joint_full_model = create_joint_model(pred_df_throughput)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_throughput, y=y_train_rapport, epochs=32, 
                    batch_size=64, 
                    validation_data=(pred_df_throughput_test,y_test_rapport), callbacks=[callback])

Joint full model with one hot encoding, predicting rapport, weighted by throughput
Model: "functional_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 3)]               0         
_________________________________________________________________
dense_30 (Dense)             (None, 2)                 8         
_________________________________________________________________
dense_31 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_32 (Dense)             (None, 2)                 10        
Total params: 30
Trainable params: 30
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
7/7 [==============================] - 0s 36ms/step - loss: 0.6969 - acc: 0.1425 - f1_m: 0.9277 - precision_m: 0.8661 - recall_m: 1.0000 - val_loss: 0.6963 - val

Epoch 30/32
7/7 [==============================] - 0s 5ms/step - loss: 0.6934 - acc: 0.6000 - f1_m: 0.9242 - precision_m: 0.8594 - recall_m: 1.0000 - val_loss: 0.6934 - val_acc: 0.5600 - val_f1_m: 0.9328 - val_precision_m: 0.8741 - val_recall_m: 1.0000
Epoch 31/32
7/7 [==============================] - 0s 6ms/step - loss: 0.6934 - acc: 0.6000 - f1_m: 0.9273 - precision_m: 0.8661 - recall_m: 1.0000 - val_loss: 0.6934 - val_acc: 0.5600 - val_f1_m: 0.9328 - val_precision_m: 0.8741 - val_recall_m: 1.0000
Epoch 32/32
7/7 [==============================] - 0s 6ms/step - loss: 0.6933 - acc: 0.6000 - f1_m: 0.9275 - precision_m: 0.8661 - recall_m: 1.0000 - val_loss: 0.6934 - val_acc: 0.5600 - val_f1_m: 0.9328 - val_precision_m: 0.8741 - val_recall_m: 1.0000


In [54]:
joint_predict = joint_full_model.predict(pred_df_throughput_test)
joint_predict_round = []
for a in joint_predict:
    joint_predict_round.append([np.argmax(a)])
precision_recall_fscore_support(y_test_rapport, np.array(joint_predict_round), average='macro')

(0.5526210484193678, 0.6162687886825817, 0.48574100046750823, None)

## WorkTime only

### Joint Model with one hot encoding 

### Joint Model with one hot encoding 

In [158]:
def create_joint_model(df):
    inputB = Input(shape=(df.shape[1],))
    c = Dense(2, activation='relu')(inputB)
    c = Dense(4, activation='relu')(c)
    c = Dense(2, activation='softmax')(c)
    full_model = Model(inputs=inputB, outputs=c)

    full_model.compile(loss='binary_crossentropy', optimizer='adam', 
                          metrics=['acc',f1_m,precision_m, recall_m])
    
    return full_model

In [159]:
print('Joint full model with one hot encoding, predicting deception, weighted by throughput')
joint_full_model = create_joint_model(pred_df_full_worktime)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_full_worktime, y=y_train_deception, epochs=32, 
                    batch_size=64, 
                    validation_data=(pred_df_full_worktime_test,y_test_deception), callbacks=[callback])

Joint full model with one hot encoding, predicting deception, weighted by throughput
Model: "functional_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 4)]               0         
_________________________________________________________________
dense_33 (Dense)             (None, 2)                 10        
_________________________________________________________________
dense_34 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_35 (Dense)             (None, 2)                 10        
Total params: 32
Trainable params: 32
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6932 - acc: 0.6332 - f1_m: 0.9754 - precision_m: 0.9523 - recall_m: 1.0000 - val_loss: 0.6931 

In [160]:
y_test_rapport = np.asarray(y_test_rapport)
y_train_rapport = np.asarray(y_train_rapport)


In [161]:
print('Joint full model with one hot encoding, predicting rapport, weighted by throughput')
joint_full_model = create_joint_model(pred_df_worktime)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_worktime, y=y_train_rapport, epochs=32, 
                    batch_size=64, 
                    validation_data=(pred_df_worktime_test,y_test_rapport), callbacks=[callback])

Joint full model with one hot encoding, predicting rapport, weighted by throughput
Model: "functional_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 3)]               0         
_________________________________________________________________
dense_36 (Dense)             (None, 2)                 8         
_________________________________________________________________
dense_37 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_38 (Dense)             (None, 2)                 10        
Total params: 30
Trainable params: 30
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6932 - acc: 0.2500 - f1_m: 0.9244 - precision_m: 0.8611 - recall_m: 0.9999 - val_loss: 0.6931 - 